###  project-01@project-001:~/spark$ gsutil -m cp -r eda_final.parquet/ gs://root-bucket-1007/pq


### gsutil cp gs://hadoop-lib/gcs/gcs-connector-hadoop3-2.2.5.jar .


In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.context import SparkContext

In [2]:
credentials_location = 'project-001.json'

conf = SparkConf() \
    .setMaster('local[*]') \
    .setAppName('test') \
    .set("spark.jars", "./lib/gcs-connector-hadoop3-2.2.5.jar") \
    .set("spark.hadoop.google.cloud.auth.service.account.enable", "true") \
    .set("spark.hadoop.google.cloud.auth.service.account.json.keyfile", credentials_location)

In [3]:
sc = SparkContext(conf=conf)

hadoop_conf = sc._jsc.hadoopConfiguration()

hadoop_conf.set("fs.AbstractFileSystem.gs.impl",  "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
hadoop_conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
hadoop_conf.set("fs.gs.auth.service.account.json.keyfile", credentials_location)
hadoop_conf.set("fs.gs.auth.service.account.enable", "true")

24/12/30 14:25:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
spark = SparkSession.builder \
    .config(conf=sc.getConf()) \
    .getOrCreate()

In [5]:
data = spark.read.parquet('gs://root-bucket-1007/pq/*/*')

In [7]:
data.show(5)

+----------------+-------------+-------------------+-------------------+--------------------+----------------+--------------------+--------------+---------+---------+-------------+---------------+---------------+------------------+-----------+-----------+
|         ride_id|rideable_type|         started_at|           ended_at|  start_station_name|start_station_id|    end_station_name|end_station_id|start_lat|start_lng|member_casual|end_lat_imputed|end_lng_imputed|     ride_duration|time_of_day|day_of_week|
+----------------+-------------+-------------------+-------------------+--------------------+----------------+--------------------+--------------+---------+---------+-------------+---------------+---------------+------------------+-----------+-----------+
|0A1B623926EF4E16|  docked_bike|2021-07-02 14:44:36|2021-07-02 15:19:58|Michigan Ave & Wa...|           13001|Halsted St & Nort...|  KA1504000117|41.883984|-87.62469|       casual|       41.89937|      -87.64848| 35.36666666666667| 